# Serial Optimization
How do you optimise serial code? There are two main techniques that are easy go-to's for serial optimization in Python: 
1. Vectorization
2. Memoization

## Vectorization
Because Python is an interpreted language - i.e. each line is dispatched to an interpreter program to interpret and execute - there can be a lot of overhead compared to compiled programs. Python needs to check variable types and use the correct functions for the inputs, because these are not necessarily specified. As a result, when you're using Python for large datasets or for long loops, it can pay to implement vectorization.

Vectorization essentially uses compiled versions of the loop or functions. There is therefore much less overhead per operation, providing large speedups. As an added benefit, they can use Single Instruction, Multiple Data (SIMD) instructions - these allow the same instruction to be performed on multiple cells of data simultaneously, providing an even greater speedup.

Consider the following examples. The first uses base Python to generate a list of integers using a for loop. The second does the same, but uses `numpy` to vectorize the calculation. The %%timeit readouts show a large speed increase in the second example.

In [45]:
def double_ints():
  result = []
  for x in range(1_000_000):
    result.append(x * 2)
  return result


In [46]:
%timeit double_ints()

36.2 ms ± 711 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
import numpy as np

In [47]:
def np_double_ints():
  ints = np.arange(1_000_000)
  return ints * 2

In [48]:
assert double_ints() == np_double_ints().tolist(), "Results do not match!"

In [49]:
%timeit np_double_ints()

1.72 ms ± 28.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


We see a large speed increase in the second example, which uses `numpy` to vectorize the calculation.

The following code, which calculates pi using Monte Carlo integration, is provided. Answer the following questions:
1. Is it correct?
2. How long does it take to run?
3. Are there any bottlenecks?

Then:
1. Vectorize the code,
2. Verify the outputs
3. Measure any speedup.

In [1]:
import random

def monte_carlo_pi(num_samples):
    inside_circle = 0
    for _ in range(num_samples):
        x, y = random.uniform(-1, 1), random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            inside_circle += 1
    return (inside_circle / num_samples) * 4

In [2]:
%timeit monte_carlo_pi(1_000_000)

306 ms ± 3.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%prun monte_carlo_pi(1_000_000)

         4000517 function calls (4000510 primitive calls) in 0.885 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2000000    0.373    0.000    0.532    0.000 random.py:498(uniform)
        1    0.309    0.309    0.776    0.776 3916854312.py:3(monte_carlo_pi)
  2000000    0.160    0.000    0.160    0.000 {method 'random' of '_random.Random' objects}
       13    0.027    0.002    0.067    0.005 socket.py:623(send)
        1    0.006    0.006    0.019    0.019 decorator.py:232(fun)
        1    0.003    0.003    0.006    0.006 {method 'execute' of 'sqlite3.Connection' objects}
        1    0.003    0.003    0.006    0.006 history.py:1008(_writeout_input_cache)
      1/0    0.003    0.003    0.000          {method 'control' of 'select.kqueue' objects}
        1    0.002    0.002    0.006    0.006 socket.py:700(send_multipart)
        3    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        1    0.000    

In [8]:
def monte_carlo_pi_numpy(num_samples):
  x, y = np.random.uniform(-1, 1, (2, num_samples))
  pi = 4 * np.mean(x**2 + y**2 <= 1)
  return pi


In [9]:
%timeit monte_carlo_pi_numpy(1_000_000)

10.7 ms ± 125 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Memoization
Memoization is a technique to cache results when they're calculated so they can be quickly retrieved when the same input is called. It avoids expensive computation of the same results again and again.

We can implement a cache ourselves using basic Python, using a simple dictionary. 

In [ ]:
cache = {}
def fib_cache(n):
    if n in cache:
        return cache[n]
    if n < 2:
        result = n
    else:
        result = fib(n-1) + fib(n-2)
    cache[n] = result
    return result


We can also use the `functools` module, which comes with Python and provides the function decorator `@lru_cache`. LRU stands for Least Recently Used - when the cache hits its size limit it removes the least recently used data to make room for the new data. 

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib_lru(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)